In [ ]:
!pip install librosa soundfile gradio opencv-python-headless deepface


In [ ]:
import os
import random
import numpy as np
import librosa
import soundfile as sf
import gradio as gr
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from deepface import DeepFace


25-12-26 04:02:18 - Directory /root/.deepface has been created
25-12-26 04:02:18 - Directory /root/.deepface/weights has been created


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/genres"


In [ ]:
mood_mapping = {
    "happy": ["pop", "disco", "reggae"],
    "sad": ["blues", "classical"],
    "energetic": ["rock", "metal", "hiphop"],
    "calm": ["jazz", "country"]
}


In [ ]:
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, duration=30)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        return np.mean(mfcc.T, axis=0)
    except:
        return None


In [ ]:
features = []
labels = []

for mood, genres in mood_mapping.items():
    for genre in genres:
        genre_path = os.path.join(DATASET_PATH, genre)

        if not os.path.exists(genre_path):
            continue

        files = [f for f in os.listdir(genre_path) if f.endswith(".au")]
        print(f"{genre}: {len(files)} files")

        for file in files:
            path = os.path.join(genre_path, file)
            feat = extract_features(path)
            if feat is not None:
                features.append(feat)
                labels.append(mood)

print("Total samples loaded:", len(features))


hiphop: 100 files
disco: 100 files
classical: 100 files
metal: 100 files
rock: 100 files
jazz: 100 files
blues: 100 files
reggae: 100 files
Total samples loaded: 800


In [ ]:
X = np.array(features)

le = LabelEncoder()
y = le.fit_transform(labels)

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=25,
    random_state=42
)

model.fit(X, y)
print(" Random Forest trained successfully")


 Random Forest trained successfully


In [ ]:
def detect_mood_from_selfie(image):
    try:
        result = DeepFace.analyze(
            img_path=image,
            actions=['emotion'],
            enforce_detection=False
        )

        emotion = result[0]['dominant_emotion']

        if emotion == "happy":
            return "happy"
        elif emotion == "sad":
            return "sad"
        elif emotion in ["angry", "fear", "disgust"]:
            return "angry"
        else:
            return "calm"

    except Exception as e:
        print("DeepFace error:", e)
        return "calm"


In [ ]:
def get_playlist_for_mood(mood):
    playlist = []

    for genre in mood_mapping[mood]:
        folder = os.path.join(DATASET_PATH, genre)
        files = [f for f in os.listdir(folder) if f.endswith(".au")]

        for f in files:
            playlist.append(os.path.join(folder, f))

    random.shuffle(playlist)
    return playlist


In [ ]:
def play_song_by_index(playlist, index):
    index = index % len(playlist)
    path = playlist[index]

    data, sr = sf.read(path)
    out = "/content/output.wav"
    sf.write(out, data, sr)

    return out, os.path.basename(path), index


In [ ]:
def start_jukebox(image):
    mood = detect_mood_from_selfie(image)

    playlist = get_playlist_for_mood(mood)
    audio, song_name, index = play_song_by_index(playlist, 0)

    return (
        f"Mood Detected: {mood}",
        audio,
        song_name,
        playlist,
        index
    )


In [ ]:
def next_song(playlist, index):
    audio, song_name, new_index = play_song_by_index(playlist, index + 1)
    return audio, song_name, new_index

def previous_song(playlist, index):
    audio, song_name, new_index = play_song_by_index(playlist, index - 1)
    return audio, song_name, new_index


In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("## 🎧 Mood Awakening Jukebox")

    image = gr.Image(type="numpy", label="Take a Selfie")

    mood_box = gr.Textbox(label="Detected Mood")
    audio = gr.Audio(label="Playing Song")
    song_name = gr.Textbox(label="GTZAN File")

    playlist_state = gr.State([])
    index_state = gr.State(0)

    start_btn = gr.Button("▶️ Start Music")
    prev_btn = gr.Button("⏮️ Previous")
    next_btn = gr.Button("⏭️ Next")

    start_btn.click(
        start_jukebox,
        inputs=image,
        outputs=[mood_box, audio, song_name, playlist_state, index_state]
    )

    next_btn.click(
        next_song,
        inputs=[playlist_state, index_state],
        outputs=[audio, song_name, index_state]
    )

    prev_btn.click(
        previous_song,
        inputs=[playlist_state, index_state],
        outputs=[audio, song_name, index_state]
    )

ui.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://97f027da1fc6cc071e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


25-12-26 04:07:05 - 🔗 facial_expression_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5 to /root/.deepface/weights/facial_expression_model_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 11.9MB/s]
